# Sprawozdanie 2
## Jakub Ciągło (275986), Mateusz Ćwiek (276011)

___
# Importy

In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportion_confint
import math

___
# Część 1
## Zadanie 1

In [3]:
def clopper_pearson_ci(successes, trials, alpha):
    lower = stats.beta.ppf(alpha / 2, successes, trials - successes + 1)
    upper = stats.beta.ppf(1 - alpha / 2, successes + 1, trials - successes)
    return lower, upper

def wyznacz_przedzialy_ufnosci(liczebnosci, alpha=0.05):
    n = np.sum(liczebnosci)
    kategorie = len(liczebnosci)
    
    adjusted_alpha = alpha / kategorie

    przedzialy = []

    for i, sukcesy in enumerate(liczebnosci):
        cp_lower, cp_upper = clopper_pearson_ci(sukcesy, n, adjusted_alpha)

        wilson_lower, wilson_upper = proportion_confint(sukcesy, n, alpha=adjusted_alpha, method='wilson')

        przedzialy.append({
            'kategoria': i + 1,
            'dokladny': (cp_lower, cp_upper),
            'asymptotyczny': (wilson_lower, wilson_upper)
        })

    return przedzialy

liczebnosci = np.array([14, 17, 40, 100, 29])
przedzialy = wyznacz_przedzialy_ufnosci(liczebnosci, alpha=0.05)

for przedzial in przedzialy:
    print(f"Kategoria {przedzial['kategoria']}")
    print(f"  Metoda Clopper-Pearson: ({przedzial['dokladny'][0]:.3f}, {przedzial['dokladny'][1]:.3f})")
    print(f"  Metoda Wilson:          ({przedzial['asymptotyczny'][0]:.3f}, {przedzial['asymptotyczny'][1]:.3f})\n")


Kategoria 1
  Metoda Clopper-Pearson: (0.032, 0.130)
  Metoda Wilson:          (0.036, 0.132)

Kategoria 2
  Metoda Clopper-Pearson: (0.042, 0.149)
  Metoda Wilson:          (0.047, 0.150)

Kategoria 3
  Metoda Clopper-Pearson: (0.133, 0.282)
  Metoda Wilson:          (0.137, 0.282)

Kategoria 4
  Metoda Clopper-Pearson: (0.407, 0.593)
  Metoda Wilson:          (0.410, 0.590)

Kategoria 5
  Metoda Clopper-Pearson: (0.087, 0.220)
  Metoda Wilson:          (0.092, 0.221)



___
## Zadanie 2

In [4]:
def poziomy_krytyczne(x, p_0):
    k = len(p_0)
    n = sum(x)
    
    pearson_stat = sum(((x[i] - n * p_0[i]) ** 2) / (n * p_0[i]) for i in range(k))

    nw_stat = 2 * sum(x[i] * np.log(x[i] / (n * p_0[i])) for i in range(k) if x[i] > 0)
    
    p_value_pearson = 1 - stats.chi2.cdf(pearson_stat, k - 1)
    p_value_nw = 1 - stats.chi2.cdf(nw_stat, k - 1)
    
    return float(p_value_pearson), float(p_value_nw)

x = [20, 20, 20, 20]
p = [0.25, 0.25, 0.25, 0.25]
print(poziomy_krytyczne(x, p))

(1.0, 1.0)


___
## Zadanie 3

In [5]:
alfa = 0.05
df = pd.read_csv('ankieta.csv', encoding='Latin2', sep=';')

df_produktowy = df[df['DZIAŁ'] == 'PD']

odpowiedzi = [-2, -1, 0, 1, 2]
n = df_produktowy['PYT_1'].value_counts().reindex(odpowiedzi, fill_value=0).tolist()
print('Liczności odpowiedzi:', n)

p_0 = [1/len(n) for _ in range(len(n))] # w hipotezie zakładamy, że prawdopodobieństwa są równe
p_value_pearson, p_value_nw = poziomy_krytyczne(n,p_0)

p_value_pearson, p_value_nw = poziomy_krytyczne(n, p_0)

print(f'Statystyka chi² Pearsona (dla poziomu istotności α = {alfa}):')
print(f'  p-wartość = {p_value_pearson}')
if p_value_pearson > alfa:
    print('  → Brak podstaw do odrzucenia hipotezy zerowej (rozkład może być równomierny)')
else:
    print('  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)')

print(f'\nStatystyka chi² największej wiarygodności (dla poziomu istotności α = {alfa}):')
print(f'  p-wartość = {p_value_nw}')
if p_value_nw > alfa:
    print('  → Brak podstaw do odrzucenia hipotezy zerowej (rozkład może być równomierny)')
else:
    print('  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)')

Liczności odpowiedzi: [9, 10, 17, 51, 11]
Statystyka chi² Pearsona (dla poziomu istotności α = 0.05):
  p-wartość = 2.757793993168889e-13
  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)

Statystyka chi² największej wiarygodności (dla poziomu istotności α = 0.05):
  p-wartość = 1.0701994845874196e-10
  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)


___
## Zadanie 5

In [6]:
# 2. Budowanie tabeli 2×2
# Wartości w kolumnach: 'PŁEĆ' (K/M), 'CZY_KIER' (Tak/Nie)
contingency_table = pd.crosstab(df['PŁEĆ'], df['CZY_KIER'])
print(contingency_table)

# 3. Test Fishera
oddsratio, p_value = stats.fisher_exact(contingency_table)

print("Wynik testu Fishera:")
print(f"  odds ratio: {oddsratio}")
print(f"  p-value: {p_value}")

alpha = 0.05
if p_value < alpha:
    print("Odrzucamy hipotezę zerową -> zmienne 'PŁEĆ' i 'CZY_KIER' są zależne.")
else:
    print("Brak podstaw do odrzucenia H0 -> nie ma dowodów na zależność.")


CZY_KIER  Nie  Tak
PŁEĆ              
K          63    8
M         110   19
Wynik testu Fishera:
  odds ratio: 1.3602272727272726
  p-value: 0.6659028889666552
Brak podstaw do odrzucenia H0 -> nie ma dowodów na zależność.


Jeśli p‑value < 0.05, odrzucamy hipotezę o niezależności, co oznacza, że istnieje istotna statystycznie różnica w prawdopodobieństwie bycia na stanowisku kierowniczym między kobietami a mężczyznami.

Jeśli p‑value ≥ 0.05 - tak jak w wyniku zadania, mamy brak podstaw do odrzucenia hipotezy zerowej, czyli nie wykryto istotnej różnicy – wtedy możemy uznać, że dane nie przeczą temu, iż prawdopodobieństwo kierowniczego stanowiska u kobiet jest podobne jak u mężczyzn.

___
## Zadanie 6

To jest raczej źle.

In [7]:
from itertools import product
from math import comb, log
import random

def freeman_halton_test(observed, n_permutations=100_000, seed=42):
    """
    Uproszczona implementacja testu Freedmana-Haltona dla tabeli m×n.
    Parametry:
      observed: 2D array-like (m×n), tabela kontyngencji
      n_permutations: liczba permutacji w symulacji
      seed: seed generatora liczb losowych (dla powtarzalności)

    Zwraca:
      stat_observed: statystyka z oryginalnej tabeli
      p_value: p-wartość obliczona permutacyjnie
    """
    np.random.seed(seed)

    observed = np.array(observed, dtype=int)
    row_sums = observed.sum(axis=1)
    col_sums = observed.sum(axis=0)
    total = observed.sum()

    # Funkcja licząca tzw. 'pseudoprawdopodobieństwo' (analogiczna do Fishera) 
    # P(table | row_sums, col_sums) ~ (prod factorial) / factorial(total)
    # ale wystarczy porównać log-prob
    def log_multinomial(table):
        # log(P) ~ sum(logfactorial(table_ij)) minus logfactorial(row_sums, col_sums, total)
        # wystarczy część licznika, bo mianownik jest stały przy fixed margins
        # W praktyce raczej liczymy sum(logfactorial(x_ij)), itp.

        # Lepsza implementacja:
        val = 0
        for i in range(table.shape[0]):
            for j in range(table.shape[1]):
                x = table[i, j]
                # logfactorial(x)
                val += math.lgamma(x + 1)
        return val

    # Oryginalna "statystyka" (log-likelihood macierzy)
    stat_observed = log_multinomial(observed)

    # Funkcja do budowy losowej tabeli z takimi samymi sumami brzegowymi
    # (row_sums i col_sums), np. metodą pogram-lan lub biprocesu
    # Tu używamy prostej metody iterative proportional fitting albo
    # sample z fisher FreedmanHalton. Poniżej skrócona, orientacyjna.
    def random_table(row_sums, col_sums):
        # Prosty algorytm step-by-step (nieoptymalny), wrzucanie komórek
        # Minimalny prymitywny approach:
        m = len(row_sums)
        n = len(col_sums)
        table = np.zeros((m, n), dtype=int)
        rows_left = row_sums.copy()
        cols_left = col_sums.copy()

        for i in range(m-1):
            for j in range(n-1):
                # Maks. co możemy wstawić to min(rows_left[i], cols_left[j])
                x = np.random.randint(0, min(rows_left[i], cols_left[j]) + 1)
                table[i,j] = x
                rows_left[i] -= x
                cols_left[j] -= x
            # Ostatnia kolumna w wierszu i
            table[i, n-1] = rows_left[i]
            cols_left[n-1] -= rows_left[i]
            rows_left[i] = 0

        # Ostatni wiersz
        for j in range(n-1):
            table[m-1, j] = cols_left[j]
            rows_left[m-1] -= cols_left[j]
            cols_left[j] = 0
        table[m-1, n-1] = rows_left[m-1]
        return table

    # Liczymy ile permutacji daje statystykę <= stat_observed (bo test dwustronny analogicznie).
    count_better = 0
    for _ in range(n_permutations):
        rand_tab = random_table(row_sums, col_sums)
        stat_rand = log_multinomial(rand_tab)
        if stat_rand <= stat_observed:
            count_better += 1

    p_value = count_better / n_permutations
    return stat_observed, p_value


In [8]:
df['WIEK_KAT'] = pd.cut(df['WIEK'], 
                        bins=[0, 35, 45, 55, float('inf')], 
                        labels=['do 35 lat', '36-45 lat', '46-55 lat', 'powyżej 55 lat'],
                        right=True)

In [9]:
import math

In [10]:
def test_freeman_halton_independence(df, var1, var2, alpha=0.05, n_permutations=2000):
    """
    Weryfikuje hipotezę niezależności var1 i var2 w DataFrame df
    na poziomie istotności alpha, używając testu Freedmana-Haltona.
    Zwraca p_value i komunikat -> 'Odrzucamy H0' lub 'Brak podstaw do odrzucenia H0'.
    """
    # Budujemy tabelę kontyngencji
    contingency = pd.crosstab(df[var1], df[var2])
    observed = contingency.values
    
    # Wywołanie testu Freedmana-Haltona (kod poglądowy)
    stat_obs, p_value = freeman_halton_test(observed, n_permutations=n_permutations)
    
    if p_value < alpha:
        decision = f"Odrzucamy H0: {var1} i {var2} są zależne."
    else:
        decision = f"Brak podstaw do odrzucenia H0: nie wykryto zależności między {var1} i {var2}."
    return p_value, decision

# --- ZASTOSOWANIE ---

alpha = 0.05

# a) CZY_KIER vs. WIEK_KAT
pval_a, dec_a = test_freeman_halton_independence(df, 'CZY_KIER', 'WIEK_KAT', alpha)
print("(a) Test Freedmana-Haltona:", pval_a, dec_a)

# b) CZY_KIER vs. STAZ
pval_b, dec_b = test_freeman_halton_independence(df, 'CZY_KIER', 'STAZ', alpha)
print("(b) Test Freedmana-Haltona:", pval_b, dec_b)

# c) PYT_2 vs. CZY_KIER
pval_c, dec_c = test_freeman_halton_independence(df, 'PYT_2', 'CZY_KIER', alpha)
print("(c) Test Freedmana-Haltona:", pval_c, dec_c)

# d) PYT_2 vs. STAZ
pval_d, dec_d = test_freeman_halton_independence(df, 'PYT_2', 'STAZ', alpha)
print("(d) Test Freedmana-Haltona:", pval_d, dec_d)

# e) PYT_2 vs. PŁEĆ
pval_e, dec_e = test_freeman_halton_independence(df, 'PYT_2', 'PŁEĆ', alpha)
print("(e) Test Freedmana-Haltona:", pval_e, dec_e)

# f) PYT_2 vs. WIEK_KAT
pval_f, dec_f = test_freeman_halton_independence(df, 'PYT_2', 'WIEK_KAT', alpha)
print("(f) Test Freedmana-Haltona:", pval_f, dec_f)

# Następnie powtarzamy c), d), e), f), zastępując PYT_2 przez CZY_ZADOW
# Najpierw w df definiujemy:
# df['CZY_ZADOW'] = df['PYT_2'].replace({
#     -2: 'niezadowolony', -1: 'niezadowolony', 1: 'zadowolony', 2: 'zadowolony'
# })

# c') CZY_ZADOW vs. CZY_KIER
pval_cz_c, dec_cz_c = test_freeman_halton_independence(df, 'CZY_ZADOW', 'CZY_KIER', alpha)
print("(c') Freedman-Halton z CZY_ZADOW:", pval_cz_c, dec_cz_c)

# d') CZY_ZADOW vs. STAZ
pval_cz_d, dec_cz_d = test_freeman_halton_independence(df, 'CZY_ZADOW', 'STAZ', alpha)
print("(d') Freedman-Halton z CZY_ZADOW:", pval_cz_d, dec_cz_d)

# e') CZY_ZADOW vs. PŁEĆ
pval_cz_e, dec_cz_e = test_freeman_halton_independence(df, 'CZY_ZADOW', 'PŁEĆ', alpha)
print("(e') Freedman-Halton z CZY_ZADOW:", pval_cz_e, dec_cz_e)

# f') CZY_ZADOW vs. WIEK_KAT
pval_cz_f, dec_cz_f = test_freeman_halton_independence(df, 'CZY_ZADOW', 'WIEK_KAT', alpha)
print("(f') Freedman-Halton z CZY_ZADOW:", pval_cz_f, dec_cz_f)

ValueError: math domain error

próba nr 2

In [11]:
from math import comb
import itertools

def hypergeom_pmf(table):
    """
    Oblicza dokładną wartość funkcji masy prawdopodobieństwa (PMF)
    dla podanej tablicy kontyngencji (2D) w oparciu o rozkład hipergeometryczny wielowymiarowy.
    
    table: pd.DataFrame lub np.ndarray – macierz R×C.
    Zakładamy, że sumy wierszy i kolumn są z góry 'zadane'.
    """
    # table musi być np.ndarray dla uproszczenia operacji
    table = np.array(table, dtype=int)
    row_sums = table.sum(axis=1)
    col_sums = table.sum(axis=0)
    grand_total = table.sum()
    
    # Licznik - iloczyn silni wartości w komórkach
    # W testach Fishera dla 2x2 jest analogiczny wzór, tu – uogólnienie
    # Prawdopodobieństwo ~ (prod( row_sums! ) * prod( col_sums! )) / grand_total! 
    # ale z uwzględnieniem części poświęconej wartościom w komórkach.
    
    # Można zapisać to za pomocą:
    # P(X = table) = ( prod_{i,j} (comb(row_sums[i], table[i,j]) ) ) / comb(grand_total, col_sums[0], col_sums[1], ...)
    # przy zachowanych sumach brzegowych.

    # Metoda 1 (wersja "product of combs"):
    # - Licznik: iloczyn comb(row_sum_i, table[i,j]) dla każdej komórki w wierszu i
    # - Mianownik: comb(grand_total, col_sums[0], col_sums[1], ...)

    numerator = 1
    for i in range(table.shape[0]):
        # Rozkład w i-tym wierszu
        row_total = row_sums[i]
        for j in range(table.shape[1]):
            numerator *= comb(row_total, table[i,j])
            row_total -= table[i,j]

    # Mianownik - wielowymiarowy współczynnik ( multinomial ), np. comb(grand_total, c1) * comb(grand_total-c1, c2) ...
    # co można zapisać jako:
    # comb(grand_total, col_sums[0]) * comb(grand_total-col_sums[0], col_sums[1]) * ... itd.
    # lub jako:
    # denominator = math.prod( comb( sum_{k=j to end} col_sums[k], col_sums[j] ) ) ...
    # Ewentualnie można posłużyć się:
    #   multinomial = grand_total! / ( col_sums[0]! col_sums[1]! ... col_sums[C-1]! )
    # ale wtedy w liczniku też trzeba by było liczyć przez silnie.
    # Dla spójności z powyższym liczymy iteracyjnie:
    
    denominator = 1
    tmp = grand_total
    for csum in col_sums:
        denominator *= comb(tmp, csum)
        tmp -= csum
    
    return numerator / denominator

def fisher_freeman_halton(table, method="exact", n_iter=10000, random_state=42):
    """
    Test Freemana–Haltona oparty o enumerację (dokładny) lub metodę Monte Carlo
    dla tabeli R×C. Zwraca p-value.
    
    - table: pd.DataFrame lub np.ndarray z nieujemnymi całkowitymi (liczności)
    - method: 'exact' lub 'montecarlo'
    - n_iter: liczba permutacji w metodzie Monte Carlo
    - random_state: do odtworzenia losowości
    """
    np.random.seed(random_state)
    
    # Jeśli to DataFrame, konwertujemy
    if isinstance(table, pd.DataFrame):
        table = table.values
    
    row_sums = table.sum(axis=1)
    col_sums = table.sum(axis=0)
    grand_total = table.sum()
    
    # Policzymy p_obs = wartość pmf(tabela zaobserwowana)
    p_obs = hypergeom_pmf(table)
    
    # 1) Metoda EXACT (enumeracja) – UWAGA: może być niewykonalna przy zbyt dużych tabelach
    if method == "exact":
        # Tworzymy wszystkie możliwe tabele z takimi samymi sumami brzegowymi:
        # to wymaga zagnieżdżonej enumeracji, co bywa ekspresowo rosnące z rozmiarem.
        # Dla małych tabel 2x2, 2x3, 3x3 i małych sum – można próbować.
        all_tables = []
        
        # Prosty (choć może nieoptymalny) sposób: 
        # iterujemy po wszystkich możliwych kombinacjach wartości w wierszach, 
        # które sumują się do row_sums[i], ale też muszą się zgadzać z col_sums.
        # W praktyce – dość skomplikowane do napisania ręcznie dla ogólnej R×C.
        # Dla uproszczenia zostawiamy taki zarys:
        
        # Uwaga: implementacja pełnej enumeracji R×C "od zera" jest dość długa,
        # poniżej tylko szkic, dlatego w praktyce częściej wybiera się Monte Carlo
        # lub pakiety gotowe w R / zewn. biblioteki.
        
        # PRZYKŁADOWY, niekompletny ZARYS:
        raise NotImplementedError("Metoda 'exact' (pełna enumeracja) jest zarysowana, "
                                  "ale w tej wersji niezaimplementowana. "
                                  "Skorzystaj z 'montecarlo' dla większych tabel.")
    
    # 2) Metoda MONTE CARLO
    # losujemy n_iter permutacji „zgodnych brzegowo” i zliczamy, ile ma p <= p_obs
    count_le = 0
    
    for _ in range(n_iter):
        # Losujemy tablicę o tych samych sumach brzegowych,
        # np. metodą iterative proportional fitting lub prostszą losową:
        # - najprostszy (choć niezbyt efektywny) sposób to wielokrotnie tasować wiersze,
        #   sprawdzać, czy kolumny się zgadzają – w praktyce dość trudne.
        # - alternatywa: wylosować z wielowymiarowego rozkładu hipergeometrycznego
        #   "zgodną" tabelę. Poniżej sposób uproszczony (tzw. "shaking columns"):
        
        # Losujemy ciąg grand_total elementów do kolumn, a następnie układamy wierszami
        # tak, by row_sums wypadły zgodnie. Jedna z podejść to tzw. "multinomial approach".
        
        # PRZYBLIŻENIE: Tasujemy wektor z etykietami kolumn, a następnie liczymy
        #   ile wylądowało w wierszu i.
        
        # Tworzymy listę kolumn rozpisaną np. col_sums[0] razy "kolumna0", col_sums[1] razy "kolumna1", ...
        col_list = []
        for j, csum in enumerate(col_sums):
            col_list.extend([j]*csum)
        
        # Mieszamy
        np.random.shuffle(col_list)
        
        # Teraz rozkładamy to w wiersze wg row_sums:
        random_table = np.zeros_like(table)
        start = 0
        for i, rsum in enumerate(row_sums):
            # Wiersz i ma rsum elementów
            row_columns = col_list[start:start+rsum]
            start += rsum
            # zliczamy ile wypadło do każdej kolumny:
            for j in row_columns:
                random_table[i, j] += 1
        
        # Liczymy pmf:
        p_rand = hypergeom_pmf(random_table)
        if p_rand <= p_obs:
            count_le += 1
    
    p_value = count_le / n_iter
    return p_value


In [13]:
df['CZY_ZADOW'] = df['PYT_2'].replace({
    -2: 'niezadowolony',
    -1: 'niezadowolony',
     1: 'zadowolony',
     2: 'zadowolony'
})

# 3. Lista par zmiennych do testowania oraz krótki opis hipotezy
to_test = [
    # (a)
    ('CZY_KIER', 'WIEK_KAT', "a) stanowisko kier. ~ wiek"),
    # (b)
    ('CZY_KIER', 'STAŻ',     "b) stanowisko kier. ~ staż"),
    # (c)
    ('PYT_2',    'CZY_KIER', "c) zadowolenie (PYT_2) ~ stanowisko kier."),
    # (d)
    ('PYT_2',    'STAŻ',     "d) zadowolenie (PYT_2) ~ staż"),
    # (e)
    ('PYT_2',    'PŁEĆ',     "e) zadowolenie (PYT_2) ~ płeć"),
    # (f)
    ('PYT_2',    'WIEK_KAT', "f) zadowolenie (PYT_2) ~ wiek"),
]

print("=== Testy Freemana–Haltona (PYT_2) ===")
for var_x, var_y, desc in to_test:
    tab = pd.crosstab(df[var_x], df[var_y])
    pval = fisher_freeman_halton(tab, method="montecarlo", n_iter=50000)
    print(f"{desc}: p-value = {pval:.4f}")
    if pval < 0.05:
        print("   Odrzucamy H0 => zmienne są zależne (istotne stat.)\n")
    else:
        print("   Brak podstaw do odrzucenia H0 => zmienne niezależne\n")

# 4. Powtórzenie testów dla CZY_ZADOW zamiast PYT_2 w punktach c), d), e), f
to_test_czy_zadow = [
    ('CZY_ZADOW', 'CZY_KIER', "c') CZY_ZADOW ~ stanowisko kier."),
    ('CZY_ZADOW', 'STAŻ',     "d') CZY_ZADOW ~ staż"),
    ('CZY_ZADOW', 'PŁEĆ',     "e') CZY_ZADOW ~ płeć"),
    ('CZY_ZADOW', 'WIEK_KAT', "f') CZY_ZADOW ~ wiek"),
]

print("=== Testy Freemana–Haltona (CZY_ZADOW) ===")
for var_x, var_y, desc in to_test_czy_zadow:
    tab = pd.crosstab(df[var_x], df[var_y])
    pval = fisher_freeman_halton(tab, method="montecarlo", n_iter=50000)
    print(f"{desc}: p-value = {pval:.4f}")
    if pval < 0.05:
        print("   Odrzucamy H0 => zmienne są zależne (istotne stat.)\n")
    else:
        print("   Brak podstaw do odrzucenia H0 => zmienne niezależne\n")

=== Testy Freemana–Haltona (PYT_2) ===
a) stanowisko kier. ~ wiek: p-value = 0.7847
   Brak podstaw do odrzucenia H0 => zmienne niezależne

b) stanowisko kier. ~ staż: p-value = 0.0001
   Odrzucamy H0 => zmienne są zależne (istotne stat.)

c) zadowolenie (PYT_2) ~ stanowisko kier.: p-value = 0.0442
   Odrzucamy H0 => zmienne są zależne (istotne stat.)

d) zadowolenie (PYT_2) ~ staż: p-value = 0.0109
   Odrzucamy H0 => zmienne są zależne (istotne stat.)

e) zadowolenie (PYT_2) ~ płeć: p-value = 0.4764
   Brak podstaw do odrzucenia H0 => zmienne niezależne

f) zadowolenie (PYT_2) ~ wiek: p-value = 0.3189
   Brak podstaw do odrzucenia H0 => zmienne niezależne

=== Testy Freemana–Haltona (CZY_ZADOW) ===
c') CZY_ZADOW ~ stanowisko kier.: p-value = 0.8389
   Brak podstaw do odrzucenia H0 => zmienne niezależne

d') CZY_ZADOW ~ staż: p-value = 0.4078
   Brak podstaw do odrzucenia H0 => zmienne niezależne

e') CZY_ZADOW ~ płeć: p-value = 0.6599
   Brak podstaw do odrzucenia H0 => zmienne niezal